In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request

In [2]:
# Access Basketball Reference to get league leaders in win shares for each season
#specify the url
site = "https://www.basketball-reference.com/leaders/hof_prob.html"
#Query the website and return the html to the variable 'page'
page = urllib.request.urlopen(site)
#Parse the html in the 'page' variable and store it in Beautiful Soup format
soup = BeautifulSoup(page, 'lxml')

In [3]:
# Save two Hall of Fame Probability leader tables (all time and active)
alltime_table, active_table = soup.find_all('table')
alltime_table

<table class="suppress_glossary suppress_csv sortable stats_table" data-cols-to-freeze="0" id="tot"><caption>NBA/ABA Table</caption><thead><tr>
<th>Rank</th>
<th>Player</th>
<th>HoF Prob</th>
</tr></thead>
<tr>
<td>1.</td>
<td><a href="/players/a/abdulka01.html">Kareem Abdul-Jabbar</a>*
</td><td>1.0000</td></tr>
<tr>
<td>2.</td>
<td><a href="/players/j/jordami01.html">Michael Jordan</a>*
</td><td>1.0000</td></tr>
<tr>
<td>3.</td>
<td><a href="/players/r/russebi01.html">Bill Russell</a>*
</td><td>1.0000</td></tr>
<tr>
<td>4.</td>
<td><a href="/players/b/bryanko01.html">Kobe Bryant</a>
</td><td>1.0000</td></tr>
<tr>
<td>5.</td>
<td><a href="/players/c/chambwi01.html">Wilt Chamberlain</a>*
</td><td>1.0000</td></tr>
<tr>
<td>6.</td>
<td><strong><a href="/players/j/jamesle01.html">LeBron James</a>
</strong></td><td>1.0000</td></tr>
<tr>
<td>7.</td>
<td><a href="/players/d/duncati01.html">Tim Duncan</a>
</td><td>1.0000</td></tr>
<tr>
<td>8.</td>
<td><a href="/players/o/onealsh01.html">Shaqui

In [4]:
active_table

<table class="suppress_glossary suppress_csv sortable stats_table" data-cols-to-freeze="0" id="stats_active_hof_prob"><caption>Active Table</caption><thead><tr>
<th>Rank</th>
<th>Player</th>
<th>HoF Prob</th>
</tr></thead>
<tr>
<td>1.</td>
<td><strong><a href="/players/j/jamesle01.html">LeBron James</a>
</strong></td><td>1.0000</td>
</tr>
<tr>
<td>2.</td>
<td><strong><a href="/players/w/wadedw01.html">Dwyane Wade</a>
</strong></td><td>1.0000</td>
</tr>
<tr>
<td>3.</td>
<td><strong><a href="/players/n/nowitdi01.html">Dirk Nowitzki</a>
</strong></td><td>1.0000</td>
</tr>
<tr>
<td>4.</td>
<td><strong><a href="/players/d/duranke01.html">Kevin Durant</a>
</strong></td><td>1.0000</td>
</tr>
<tr>
<td>5.</td>
<td><strong><a href="/players/p/paulch01.html">Chris Paul</a>
</strong></td><td>0.9999</td>
</tr>
<tr>
<td>6.</td>
<td><strong><a href="/players/c/curryst01.html">Stephen Curry</a>
</strong></td><td>0.9987</td>
</tr>
<tr>
<td>7.</td>
<td><strong><a href="/players/w/westbru01.html">Russell

In [5]:
#create list tuples of players and href 
alltime_list = []
active_list = []
for line in alltime_table('td'):
    try:
        alltime_list.append((str(line.a.string), str(line.a.get('href'))))
    except:
        pass
    
for line in active_table('td'):
    try:
        active_list.append((str(line.a.string), str(line.a.get('href'))))
    except:
        pass

In [6]:
len(alltime_list), alltime_list[:10]

(250,
 [('Kareem Abdul-Jabbar', '/players/a/abdulka01.html'),
  ('Michael Jordan', '/players/j/jordami01.html'),
  ('Bill Russell', '/players/r/russebi01.html'),
  ('Kobe Bryant', '/players/b/bryanko01.html'),
  ('Wilt Chamberlain', '/players/c/chambwi01.html'),
  ('LeBron James', '/players/j/jamesle01.html'),
  ('Tim Duncan', '/players/d/duncati01.html'),
  ("Shaquille O'Neal", '/players/o/onealsh01.html'),
  ('John Havlicek', '/players/h/havlijo01.html'),
  ('Oscar Robertson', '/players/r/roberos01.html')])

In [7]:
len(active_list), active_list[:10]

(100,
 [('LeBron James', '/players/j/jamesle01.html'),
  ('Dwyane Wade', '/players/w/wadedw01.html'),
  ('Dirk Nowitzki', '/players/n/nowitdi01.html'),
  ('Kevin Durant', '/players/d/duranke01.html'),
  ('Chris Paul', '/players/p/paulch01.html'),
  ('Stephen Curry', '/players/c/curryst01.html'),
  ('Russell Westbrook', '/players/w/westbru01.html'),
  ('James Harden', '/players/h/hardeja01.html'),
  ('Dwight Howard', '/players/h/howardw01.html'),
  ('Carmelo Anthony', '/players/a/anthoca01.html')])

In [8]:
# Create list of players on active_list but not alltime_list
# and add to alltime_list
active_list = list(set(active_list).difference(set(alltime_list)))
alltime_list += active_list
# Print new list length and last 10 players of list
print(len(alltime_list))
print(alltime_list[-10:])

310
[('Nikola Vucevic', '/players/v/vucevni01.html'), ('Sasha Vujacic', '/players/v/vujacsa01.html'), ('Brook Lopez', '/players/l/lopezbr01.html'), ('Gordon Hayward', '/players/h/haywago01.html'), ('Mario Chalmers', '/players/c/chalmma01.html'), ('Luol Deng', '/players/d/denglu01.html'), ('Andrew Bogut', '/players/b/bogutan01.html'), ('J.J. Redick', '/players/r/redicjj01.html'), ('George Hill', '/players/h/hillge01.html'), ('Joakim Noah', '/players/n/noahjo01.html')]


In [9]:
# This for loop gets the season by season stats for each player in the 
# alltime_list. First it turns the player page html into a Beautiful Soup
# object to make the stats table. I also pull the player's height from
# the top of the page, and convert it from a string 'x-xx' to inches.
all_seasons = [] 
for player in alltime_list[:20]:
    reference_site = 'https://www.basketball-reference.com' + player[1]
    page = urllib.request.urlopen(reference_site)
    soup = BeautifulSoup(page, 'lxml')
    per_game_table = soup.table
    
    height = str(soup.find_all('div', {'id':'info'})[0].find_all('span', {'itemprop':'height'})[0].string)
    height = height_str.split('-')
    height = int(height[0])*12 + int(height[1])
    
    # This section gets stats values for a single season and 
    # makes a single list
    values = []
    length = 0
    for row in (per_game_table('tr')):
        for num, column in enumerate(row):
            # Players from different decades have different
            # available stats, but PTS is always the last column
            if column.string == 'PTS':
                length = int((num+1) / 2)
            if column != '\n':
                values.append(str(column.string))
    categories = values[:length]
    # Ignores category names
    values = values[length:]

    # Create list of individual season stats lists
    player_career = [['Player', 'href', 'Height'] + categories]
    season = [player[0], player[1], height]
    for value in values:
        if value.startswith('Did'):
            # Find players who took time off mid-career
            print(player[0], value)
        season.append(str(value))
        #added and (season[3] != None)
        if (len(season) == length+3) and (season[4] != 'None'):
            player_career.append(season)
            season = [player[0], player[1], height]
        # Player must have played at least 12 seasons
    #if (len(player_career) > 12):# and (len(player_career[0]) == 33):
    all_seasons.append(player_career)
    print(str((len(player_career)-1)), ' seasons of ', player[0], 'added')


20  seasons of  Kareem Abdul-Jabbar added
Michael Jordan Did Not Play (retired)
Michael Jordan Did Not Play (retired)
Michael Jordan Did Not Play (retired)
17  seasons of  Michael Jordan added
13  seasons of  Bill Russell added
20  seasons of  Kobe Bryant added
16  seasons of  Wilt Chamberlain added
16  seasons of  LeBron James added
19  seasons of  Tim Duncan added
21  seasons of  Shaquille O'Neal added
16  seasons of  John Havlicek added
14  seasons of  Oscar Robertson added
Bob Cousy Did Not Play (retired)
Bob Cousy Did Not Play (retired)
Bob Cousy Did Not Play (retired)
Bob Cousy Did Not Play (retired)
Bob Cousy Did Not Play (retired)
Bob Cousy Did Not Play (retired)
15  seasons of  Bob Cousy added
23  seasons of  Kevin Garnett added
14  seasons of  Jerry West added
Magic Johnson Did Not Play (illness—HIV)
Magic Johnson Did Not Play (illness—HIV)
Magic Johnson Did Not Play (illness—HIV)
Magic Johnson Did Not Play (illness—HIV)
14  seasons of  Magic Johnson added
19  seasons of  Kar

In [10]:
# Create pandas DataFrame of all data
category_list= ['Player', 'href', 'Height', 'Season', 'Age', 'Tm', 'Lg', 
                'Pos', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', 
                '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 
                'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'
                ]

In [12]:
# CREATE LIST OF DATAFRAMES AND MERGE AT END
df_list = []
season_df_list = []
master_df = pd.DataFrame(columns=category_list)
for career in all_seasons:
    labels=career[0]
    for season in career[1:]:
        season_df = pd.DataFrame(data=season, index=labels)
        season_df_list.append(season_df.transpose())
    print(career[1][0] + ' added')
career_df = pd.concat(season_df_list, ignore_index=True)
    

career_df.to_csv('all-stats-sample.csv', columns = category_list)

Kareem Abdul-Jabbar added
Michael Jordan added
Bill Russell added
Kobe Bryant added
Wilt Chamberlain added
LeBron James added
Tim Duncan added
Shaquille O'Neal added
John Havlicek added
Oscar Robertson added
Bob Cousy added
Kevin Garnett added
Jerry West added
Magic Johnson added
Karl Malone added
Hakeem Olajuwon added
Dwyane Wade added
Dirk Nowitzki added
Larry Bird added
Elvin Hayes added
